<a href="https://colab.research.google.com/github/ShaharWeisberger/Search_Engine_wikipedia/blob/master/page_viewww_writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
from google.colab import drive
drive.mount('/content/drive')
import hashing as hs

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
import importlib
import hashing as hs
importlib.reload(hs)

<module 'hashing' from '/content/hashing.py'>

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
print(pv_name)
print(pv_temp)
print(pv_clean)


# !wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
# !bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})


pageviews-202108-user.bz2
pageviews-202108-user-4dedup.txt
pageviews-202108-user.pkl


In [ ]:
sort_dic = sorted(list(wid2pv.items()),key=lambda x: x[0])


with open('id_view_dict.pickle', 'wb') as f:
  pickle.dump(wid2pv, f)

In [ ]:
# sort_dic = sorted(list(wid2pv.items()),key=lambda x: x[0])
# for i in sort_dic:
#   print(type(i[0]))
#   break